In [5]:
import torch

# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
# check if cuda is available
print(torch.cuda.is_available())

True
True
False


In [6]:
import h5py
import numpy as np
import time
import timeit
import os

# import filters
import filters as ft
import torch_filters as tft

In [7]:
def stats(x):
    return x.shape, np.mean(x,axis=1), np.cov(x[:1000,:],ddof=1)

def close(state_analysis,xb):
    return np.allclose(state_analysis.mean(axis=1),xb.mean(axis=1))

def timer(filter,filter_object):
    """
    function execution time
    """
    st = time.time()
    x = filter_object._assimilate()
    et = time.time()

    # get the execution time
    elapsed_time = et - st
    
    return f"Filter: {filter.__name__} Execution time: {elapsed_time} seconds"


def timeit(filter,filter_object):
    """
    function execution time using python timeit module
    """
    %timeit filter_object._assimilate()
    


In [8]:
hf = h5py.File('data/example_sim_outputs.h5',"r")
zeta = np.array(hf.get('zeta'))
obs = np.array(hf.get('obs'))
    
print(f"Ensemble state matrix dimensions (ne x nx): {zeta.shape}")
print(f"Observation vector at one time step dimensions: (ny) {obs.shape}")


state_forecast = zeta.T[:,:5]                            # ensemble of state vectors
observations = obs                                      # model observations (adcirc output file fort.63.nc)                  
model_observations = np.eye(obs.size)[:,:5]             # observation operator ( adcirc output file fort.61.nc)

params = [state_forecast,model_observations,observations]

Ensemble state matrix dimensions (ne x nx): (10, 3352598)
Observation vector at one time step dimensions: (ny) (11,)


In [12]:
ft.Estkf.__name__.lower()

'filters.Estkf'

In [13]:
filters = [ft.Estkf,ft.Ensrf,ft.Senkf,tft.EstkfTorch]

for i, filter in enumerate(filters):
    # filter_class= getattr(filter,filter.__name__.lower())
    filter_object= filter(*params)
    print(f"{timer(filter,filter_object)}")
    timeit(filter,filter_object)
    print("\n")

Filter: Estkf Execution time: 0.22228717803955078 seconds
62.3 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Filter: Ensrf Execution time: 0.3153228759765625 seconds
254 ms ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Filter: Senkf Execution time: 0.058332204818725586 seconds
56.4 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Filter: EstkfTorch Execution time: 0.16088294982910156 seconds


AttributeError: 'Tensor' object has no attribute 'copy'

In [9]:
# filters = [ft.Estkf,ft.Ensrf,ft.Senkf,tft.Estkf_torch]

# for i, filter in enumerate(filters):
#     filter_class= getattr(filter,filter.__name__.lower())
#     filter_object= filter_class(*params)
#     print(f"{timer(filter,filter_object)}")
#     timeit(filter,filter_object)
#     print("\n")
   

AttributeError: module 'filters.Estkf' has no attribute 'filters.Estkf'

In [ ]:
# print(stats(x1),"\n")
# print(stats(x2),"\n")
# print(close(x1,x2),"\n")
